In [7]:
import pandas as pd
import numpy as np
import pickle
from pystoned import CNLS, CNLSDDF, StoNED
from pystoned.constant import CET_MULT, CET_ADDI, FUN_COST, FUN_PROD, RTS_VRS, RTS_CRS, RED_MOM, OPT_LOCAL,RED_KDE, RED_QLE 

In [2]:
data = pd.read_csv("../data_ready/data_ready.csv").replace([np.inf, -np.inf], np.nan).dropna()


In [3]:
inputs = data[["total_students", "ratio_f2m", "ratio_s2t", "ratio_ft2pt"]].values.tolist()
outputs = data[["avg_math", "avg_science", "avg_read"]]
d_outputs = data[["avg_math", "avg_science", "avg_read"]].values.tolist()
undesirables = data[["dropout_rate"]].values.tolist()
contextuals = data[["bol_competition", "bol_location"]].values.tolist()
data["avg_scores"] = outputs.mean(axis = 1)
z_output = data[["avg_scores"]].values.tolist()

In [4]:
# define the CNLS-Z model
model1 = CNLS.CNLS(
    x = inputs, 
    y = z_output, 
    z = contextuals, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_CRS
)

model2 = CNLS.CNLS(
    x = inputs, 
    y = z_output, 
    z = contextuals, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_VRS
)

# define the CNLSDDF model with undesirable output
model3 = CNLSDDF.CNLSDDF(
    x = inputs, 
    y = d_outputs, 
    b = undesirables, 
    gb = -1.0,
    gy = [1.0,1.0, 1.0],
    gx = [0,0,0,0],
    fun = FUN_PROD
)

# define the CNLSDDF model with undesirable output
model4 = CNLSDDF.CNLSDDF(
    x = inputs, 
    y = d_outputs, 
    gy = [1.0, 1.0, 1.0],
    gx = [0,0,0,0],
    fun = FUN_PROD
)
# # display the coefficient of contextual variable

In [5]:
model1.optimize(OPT_LOCAL)
model2.optimize(OPT_LOCAL)
model3.optimize(OPT_LOCAL)
model4.optimize(OPT_LOCAL)

Optimizing locally.
Estimating the additive model locally with mosek solver.
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : QO (quadratic optimization problem)
  Constraints            : 388129          
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 4363            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.18    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 623
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.

In [12]:
ineff1_all = StoNED.StoNED(model1)
ineff2_all = StoNED.StoNED(model2)
ineff3_all = StoNED.StoNED(model3)
ineff4_all = StoNED.StoNED(model4)

In [13]:
res_all = {
    "res1_all" : {
        "alpha" : "None",
        "beta" : model1.get_beta(),
        "delta" : "None",
        "gamma" : "None",
        "lamda" : model1.get_lamda(),
        "residuals" : model1.get_residual(),
        "frontier" : model1.get_frontier(),
        "unconditional_expected_inefficiency" : ineff1_all.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff1_all.get_technical_inefficiency(method = RED_QLE)
    },


    "res2_all" : {
        "alpha" : model2.get_alpha(),
        "beta" : model2.get_beta(),
        "delta" : "None",
        "gamma" : "None",
        "lamda" : model2.get_lamda(),
        "residuals" : model2.get_residual(),
        "frontier" : model2.get_frontier(),
        "unconditional_expected_inefficiency" : ineff2_all.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff2_all.get_technical_inefficiency(method = RED_QLE)
    },


    "res3_all" : {
        "alpha" : model3.get_alpha(),
        "beta" : model3.get_beta(),
        "delta" : model3.get_delta(),
        "gamma" : model3.get_gamma(),
        "lamda" : "None",
        "residuals" : model3.get_residual(),
        "frontier" : "None",
        "unconditional_expected_inefficiency" : ineff3_all.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff3_all.get_technical_inefficiency(method = RED_QLE)
    },


    "res4_all" : {
        "alpha" : "None",
        "beta" : model4.get_beta(),
        "delta" : "None",
        "gamma" : model4.get_gamma(),
        "lamda" : "None",
        "residuals" : model4.get_residual(),
        "frontier" : "None",
        "unconditional_expected_inefficiency" : ineff4_all.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff4_all.get_technical_inefficiency(method = RED_QLE)
    }
}


NameError: name 'RED_KDE' is not defined

In [8]:
import pickle
file = 'res_all'
open_file = open(file, "wb")
pickle.dump(res_all, open_file)
open_file.close()

# file2 = '../results/res_cze'
# open_file2 = open(file2, "wb")
# pickle.dump(res_cze, open_file2)
# open_file.close()

NameError: name 'res_all' is not defined